In [1]:
from keras.models import Sequential, load_model
import pandas as pd
import numpy as np
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import random
import datetime

from Indicator import Indicator

def check_trend(df):
    df.loc[(df['Close'] > df['MA']) &
    ((df['Close'] > df['Close'].shift(1)) | (df['MA'] > df['MA'].shift(1))) &
    ((df['Close'] > df['Close'].shift(2)) | (df['MA'] > df['MA'].shift(2))) &
    ((df['Close'] > df['Close'].shift(3)) | (df['MA'] > df['MA'].shift(3))) &
    ((df['Close'] > df['Close'].shift(4)) | (df['MA'] > df['MA'].shift(4))) &
    ((df['Close'] > df['Close'].shift(5)) | (df['MA'] > df['MA'].shift(5))) , 'Trend'] = 'Up'

    df.loc[(df['Close'] < df['MA']) &
    ((df['Close'] < df['Close'].shift(1)) | (df['MA'] < df['MA'].shift(1))) &
    ((df['Close'] < df['Close'].shift(2)) | (df['MA'] < df['MA'].shift(2))) &
    ((df['Close'] < df['Close'].shift(3)) | (df['MA'] < df['MA'].shift(3))) &
    ((df['Close'] < df['Close'].shift(4)) | (df['MA'] < df['MA'].shift(4))) &
    ((df['Close'] < df['Close'].shift(5)) | (df['MA'] < df['MA'].shift(5))) , 'Trend'] = 'Down'
    
    df.loc[df['Trend'].isnull(), 'Ntrend'] = df['Trend'].shift(1)
    df.loc[df['Trend'].isnull() & df['Trend'].shift(1).isnull(),'Ntrend'] = df['Trend'].shift(2)
    df.loc[df['Trend'].isnull() & df['Trend'].shift(1).isnull() & df['Trend'].shift(2).isnull(),'Ntrend'] = df['Trend'].shift(3)
    df.loc[df['Trend'].isnull() & df['Trend'].shift(1).isnull() & df['Trend'].shift(2).isnull() & 
    df['Trend'].shift(3).isnull(),'Ntrend'] = df['Trend'].shift(4)
    df.loc[df['Trend'].isnull() & df['Trend'].shift(1).isnull() & df['Trend'].shift(2).isnull() & 
    df['Trend'].shift(3).isnull() & df['Trend'].shift(4).isnull(),'Ntrend'] = df['Trend'].shift(5)
    df.loc[df['Trend'].isnull() & df['Trend'].shift(1).isnull() & df['Trend'].shift(2).isnull() & 
    df['Trend'].shift(3).isnull() & df['Trend'].shift(4).isnull() & df['Trend'].shift(5).isnull(),'Ntrend'] = df['Trend'].shift(6)
    
    df['S1'] = df['Close'].shift(-1)
    df['S2'] = df['Close'].shift(-2)
    df.loc[df['S1'].isnull(),'S1'] = 0
    df.loc[df['S2'].isnull(),'S2'] = 0

   
    #Trading Signal (0,1)
    df.loc[(df['Trend'] == 'Up') | (df['Ntrend'] == 'Up'), 'Trading Signal'] = ((abs(df['Close'] - df[['Close','S1','S2']].min(axis = 1)) / 
    abs(df[['Close','S1','S2']].max(axis = 1) - df[['Close','S1','S2']].min(axis = 1)))*0.5) + 0.5
    df.loc[(df['Trend'] == 'Down') | (df['Ntrend'] == 'Down'), 'Trading Signal'] = ((abs(df['Close'] - df[['Close','S1','S2']].min(axis = 1)) / 
    abs(df[['Close','S1','S2']].max(axis = 1) - df[['Close','S1','S2']].min(axis = 1)))*0.5)
   
    df.loc[df['Trend'].isnull(),'Trend'] =df['Ntrend']
    
    df = df.drop(['Ntrend','S1','S2'],axis=1)
        
    return df

def fillindicator(df):
    indicator = Indicator()
    train_set = indicator.MA(df,15)
    train_set = indicator.MACD(df)
    train_set = indicator.STOCHASTIC(df)
    train_set = indicator.RSI(df)
    train_set = indicator.WILLIAMSR(df)
    train_set = indicator.DMI(df)
    train_set = indicator.AROON(df)
    train_set = indicator.ATR(df)
    train_set = indicator.CCI(df)
    train_set = indicator.OBV(df)
    return train_set

def norm(df,col_name):# this will get columnname + _n
    #Normalize data each colunm in to range -1 to 1
    df[col_name+'_n'] = (df[col_name]-df[col_name].min())/(df[col_name].max()-df[col_name].min())
    return df

def signal_to_num(df):
    df.loc[df['Trend']=='Down','Trend'] = 0
    df.loc[df['Trend']=='Up','Trend'] = 1
    return df

In [2]:
start = datetime.datetime(2019,10,1)#26 day before start year for make indicator
end = datetime.datetime(2021,1,1)

stock = 'KTC.bk'

In [3]:
df = web.DataReader(stock,'yahoo',start,end)
df = fillindicator(df)
df = norm(df,'MA')
df = norm(df,'MACD')
df = norm(df,'SIGNAL LINE')
df = norm(df,'%K')
df = norm(df,'%D')
df = norm(df,'%R')
df = norm(df,'RSI-7')
df = norm(df,'RSI-14')
df = check_trend(df)
df = df.iloc[61:]
df = signal_to_num(df)

model = load_model('Model/best_5.h5')

test_x = df[['MA_n','MACD_n','SIGNAL LINE_n','RSI-7_n','%K_n','%D_n','%R_n','Trading Signal','Trend']]
x_test = test_x[['MA_n','MACD_n','SIGNAL LINE_n','RSI-7_n','%K_n','%D_n','%R_n','Trading Signal']].values
y_test = test_x['Trend'].values
x_test=np.asarray(x_test).astype(np.float32)
y_test=np.asarray(y_test).astype(np.float32)

pred = model.predict(x_test)

data = df
data['pred'] = pred
mean = data['pred'].mean()
data['pred'] =np.where(data['pred'] >= mean,'up','down')

data['up'] = np.where(data['pred']=='up',data['Close'],None)
data['down'] = np.where(data['pred']=='down',data['Close'],None)

b = []
s = []
flag = 'sell'
for i,r in data.iterrows():
    if (flag=='sell') & (r['pred']=='up'):
        b.append(r['Close'])
        flag = 'buy'
    elif (flag=='buy') & (r['pred']=='down'):
        s.append(r['Close'])
        flag = 'sell'
        
if len(b)>len(s):
    del b[-1]
    
pfmodel = sum(np.array(s)-np.array(b))

C:\Users\jame_\anaconda3\lib\site-packages\ta\trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
C:\Users\jame_\anaconda3\lib\site-packages\ta\trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


In [4]:
df = web.DataReader(stock,'yahoo',start,end)
df = fillindicator(df)
df = df.iloc[61:]

df['signal'] = 0
df.loc[((df['MACD']>df['SIGNAL LINE'])&(df['MACD'].shift()<df['SIGNAL LINE'].shift())),'signal'] = 1
df.loc[((df['MACD']<df['SIGNAL LINE'])&(df['MACD'].shift()>df['SIGNAL LINE'].shift())),'signal'] = 2

prd = []
for i ,row in df.iterrows():
    if row['signal']==1:
#         plt.scatter(x=i, y=row['Close'], c='green')
        prd.append(('buy',row['Close']))
    elif row['signal']==2:
#         plt.scatter(x=i, y=row['Close'], c='red')
        prd.append(('sell',row['Close']))

if prd[0][0]=='sell':
    del prd[0]
if len(prd) %2!=0:
    del prd[-1]

b = []
s = []
for i in prd:
    if i[0] =='buy':
        b.append(i[1])
    else:
        s.append(i[1])

pfmacd=sum(np.array(s)-np.array(b))


In [5]:
print('Profit MACD = ',pfmacd)
print('Profit Model = ',pfmodel)

Profit MACD =  31.75
Profit Model =  28.0
